In [1]:
# Importing Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Setting path to csv created from api call data
api_data = "./Outputs/active_count.csv"

# Converting csv to pandas dataframe and verifying data
player_count_df = pd.read_csv(api_data)
player_count_df.head()

,Unnamed: 0,AppID,Current Players
0,0,10,14349


In [3]:
# Defining columns of interest from imported data
count_cols = ["AppID", "Current Players"]

# Dropping columns outside of defined count_cols
count_transformed = player_count_df[count_cols].copy()
count_transformed = count_transformed.rename(columns={"AppID": "appid",
                                                      "Current Players": "current_players"})

# Dropping any potential duplicates to maintain data integrity
count_transformed.drop_duplicates("appid", inplace=True)

# Setting game AppID as index to simplify data location in case multiple get requests are performed and to simplify SQL import
count_transformed.set_index("appid", inplace=True)

# Verifying dataframe columns were transformed correctly
count_transformed.head()

,current_players
appid,
10,14349


In [4]:
# Creating SQL engine
connection_string = "postgres:postgres@localhost:5432/playercount_db"
engine = create_engine(f'postgresql://{connection_string}')

In [5]:
# Verifying tables in database
engine.table_names()

['current_players']

In [6]:
# Importing dataframe data into SQL database
count_transformed.to_sql(name='current_players', con=engine, if_exists='append', index=True)

In [ ]:
query_str = open('SQL/players_query.sql')
query_text = ""

for text in query_str:
    query_text = query_text + text

print(query_text)
player_query_df = pd.read_sql_query(query_text, con=engine)

player_query_df